# "[Drafted] -Optimizing pandas - Reducing 90% memory footprint - updated version"
> This post is base on https://www.dataquest.io/blog/pandas-big-data/ and updated with a new automated functions from https://github.com/ianozsvald/dtype_diet/blob/master/dtype_diet.py


- toc: true 
- badges: true
- comments: true
- categories: [python, pandas, optimization]
- hide: false

# Todo
- [ ] TWO options to automatically optimize pandas

In [1]:
#hide
%load_ext autoreload
%autoreload 2

import dtype_diet
from dtype_diet import *

In [2]:
#hide
import pandas as pd

# You need to download the dataset from https://data.world/dataquest/mlb-game-logs
df_gamelogs = pd.read_csv('../data/dataquest-mlb-game-logs.zip')
df_gamelogs.head()

C:\Users\lrcno\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12,13,14,15,19,20,81,83,85,87,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


We can check some basic info about the data with pandas `.info()` function

In [3]:
df_gamelogs.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 860.5 MB


We can see the data has 171907 rows and 161 columns and 859.4 MB memory. Let's see how much we can optimize `dtype_diet`.

In [4]:
proposed_df = report_on_dataframe(df_gamelogs, unit="MB")
proposed_df

,Current dtype,Proposed dtype,Current Memory (MB),Proposed Memory (MB),Ram Usage Improvement (MB),Ram Usage Improvement (%)
Column,,,,,,
date,int64,int32,671.574219,335.818359,335.755859,49.995347
number_of_game,int64,int8,671.574219,84.001465,587.572754,87.491857
day_of_week,object,category,5036.400391,84.362793,4952.037598,98.324939
v_name,object,category,5036.400391,174.776367,4861.624023,96.529736
v_league,object,category,4952.461426,84.359375,4868.102051,98.296617
...,...,...,...,...,...,...
h_player_9_id,object,category,4955.471680,412.757324,4542.714355,91.670675
h_player_9_name,object,category,5225.463379,421.197266,4804.266113,91.939523
h_player_9_def_pos,float64,float16,671.574219,167.940430,503.633789,74.993020


In [5]:
new_df = optimize_dtypes(df_gamelogs, proposed_df) # Avoid Type conversion error from int64 to int 8 with NA

In [6]:
print(f'Original df memory: {df_gamelogs.memory_usage(deep=True).sum()/1024/1024} MB')
print(f'Propsed df memory: {new_df.memory_usage(deep=True).sum()/1024/1024} MB')

Original df memory: 860.500262260437 MB
Propsed df memory: 79.04368686676025 MB


In [7]:
# sell_prices.csv.zip Source data: https://www.kaggle.com/c/m5-forecasting-uncertainty/
df = pd.read_csv('../data/sell_prices.csv.zip')

In [8]:
proposed_df = report_on_dataframe(df, unit="MB")
new_df = optimize_dtypes(df, proposed_df) # Avoid Type conversion error from int64 to int 8 with NA

In [9]:
print(f'Original df memory: {df_gamelogs.memory_usage(deep=True).sum()/1024/1024} MB')
print(f'Propsed df memory: {new_df.memory_usage(deep=True).sum()/1024/1024} MB')

Original df memory: 860.500262260437 MB
Propsed df memory: 85.09655094146729 MB


In [10]:
## hide
## collapse-hide
